# 自动播放新闻联播

In [1]:
import os
from time import sleep

In [2]:
from selenium import webdriver


## 配置 flash player 插件

In [3]:
from selenium.webdriver.chrome.options import Options

cr_options = Options()
cr_options.headless = False

## solution out-of-date
# prefs = {
#     "profile.default_content_setting_values.plugins": 1,
#     "profile.content_settings.plugin_whitelist.adobe-flash-player": 1,
#     "profile.content_settings.exceptions.plugins.*,*.per_resource.adobe-flash-player": 1,
#     "PluginsAllowedForUrls": [
#         "http://tv.cctv.com/2020/03/30/VIDEZ1dw3QAqvk3f94HmjXEb200330.shtml",
#     ]
# }

# cr_options.add_experimental_option("prefs",prefs)
cr_options.add_argument('--allow-outdated-plugins')

_home = os.getenv("HOME")
flash_player_path = f"{_home}/.config/google-chrome/PepperFlash/32.0.0.344/libpepflashplayer.so"
cr_options.add_argument('--ppapi-flash-version=32.0.0.344')
cr_options.add_argument('--ppapi-flash-path=' + flash_player_path)

In [4]:
browser = webdriver.Chrome(
    executable_path="/usr/bin/chromedriver",
    options=cr_options, )

## 启用 flash

In [5]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [6]:
browser.get("chrome://settings/content/flash")

actions = ActionChains(browser)
actions = actions.send_keys(Keys.TAB * 19)
actions = actions.send_keys(Keys.SPACE)
# actions = actions.send_keys("a")
# actions = actions.send_keys(Keys.ENTER)
actions.perform()

## 最新的新闻 30 分（昨晚 19 点）

### 百度搜索关键词“新闻联播”

In [7]:
browser.get("https://www.baidu.com/s?ie=utf-8&rsv_bp=1&ch=&bar=&wd={}".format("新闻联播"))

In [8]:
if False:
    elem = browser.find_element_by_xpath('//div[@class="c-gap-top"]')
    sel_png = elem.screenshot_as_png
    img_path = "/tmp/selenium-dbg.png"
    with open(img_path, "wb") as fp:
        fp.write(sel_png)

    os.system(f"xdg-open {img_path}")

### 获取昨日的日期

In [9]:
import datetime as dt
from datetime import datetime

In [10]:
_n = datetime.now()

In [11]:
one_day = dt.timedelta(days=1)
yesterday = _n - one_day
print(yesterday.strftime("%Y%m%d"))
_s = yesterday.strftime("%Y%m%d")

20200330


### 找到最新的播放链接

In [12]:
cctv_url = ""

In [13]:
i = 1
while True:
    try:
        elem = browser.find_element_by_xpath(
            f'//div[@class="c-gap-top"]//*//div[@class="tab-tvshow-span c-span6"][{i}]'
        )
    except Exception as err:
        print("{!r}".format(err))
        break
    else:
        #print(elem.get_attribute('innerHTML'))
        if _s in elem.get_attribute("innerHTML"):
            elem.click()
            break
        else:
            i += 1

<div class="dis-relative" data-a-7701177d=""><span class="tab-tvshow-link-newest dis-absolute" data-a-7701177d="">最新</span><a href="http://www.baidu.com/link?url=zjrgvWDhZEvWcFtKgt1pyHXD5JhD65lnDAS-V_Q55hhx0Vl6PIbUEUA6w3WsvhbP8N7MZ0wT5Q-FQIURjqFoC3x6KzQ5e4ky_u1r_ND3LdW" target="_blank" data-visited="off" class="c-blocka" data-a-7701177d="" data-a-339f6d90=""><div class="dis-relative" data-a-7701177d=""><img src="https://dss2.baidu.com/6ONYsjip0QIZ8tyhnq/it/u=3325279217,381738190&amp;fm=58&amp;app=83&amp;f=JPEG?w=300&amp;h=200&amp;s=AF01A4446820B64D56E05C810300E08E" class="c-img c-img6 tab-tvshow-link-img" data-a-7701177d=""><p class="tabs-tvshow-info dis-line-block dis-absolute" data-a-7701177d=""><i class="c-icon c-icon-play-black c-gap-left-small c-gap-right op-vd-variety-play" data-a-7701177d=""></i><span data-a-7701177d="">2020-03-31</span></p></div></a><a href="http://www.baidu.com/link?url=zjrgvWDhZEvWcFtKgt1pyHXD5JhD65lnDAS-V_Q55hhx0Vl6PIbUEUA6w3WsvhbP8N7MZ0wT5Q-FQIURjqFoC3x6KzQ

In [14]:
# cctv_url = "http://tv.cctv.com/2020/03/30/VIDEZ1dw3QAqvk3f94HmjXEb200330.shtml"

In [15]:
browser.switch_to_window(browser.window_handles[1])

/home/josephlin/.virtualenvs/fuuucknetease/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [16]:
cctv_url = browser.current_url
print(f"[DEBUG] {cctv_url}")

[DEBUG] http://tv.cctv.com/2020/03/30/VIDEZ1dw3QAqvk3f94HmjXEb200330.shtml


In [17]:
browser.close()
browser.switch_to_window(browser.window_handles[0])

/home/josephlin/.virtualenvs/fuuucknetease/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.window instead
  


## 对播放链接允许 flash

In [18]:
browser.get("chrome://settings/content/siteDetails?site=" + cctv_url)

actions = ActionChains(browser)
actions = actions.send_keys(Keys.TAB * 21)
actions = actions.send_keys(Keys.SPACE)
actions = actions.send_keys("a")
actions = actions.send_keys(Keys.ENTER)

actions = actions.send_keys(Keys.TAB)
actions = actions.send_keys(Keys.SPACE)
actions = actions.send_keys("a")
actions = actions.send_keys(Keys.ENTER)

actions = actions.send_keys(Keys.TAB)
actions = actions.send_keys(Keys.SPACE)
actions = actions.send_keys("a")
actions = actions.send_keys(Keys.ENTER)

actions.perform()
# sleep(5)

In [19]:
## this solution is not good enough
if False:
    browser.get("chrome://settings/content/siteDetails?site=" + cctv_url)
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import Select

    def expand_root_element(driver, element):
        return driver.execute_script("return arguments[0].shadowRoot", element)

    root1 = browser.find_element(By.TAG_NAME, "settings-ui")
    shadow_root1 = expand_root_element(browser, root1)
    root2 = shadow_root1.find_element(By.ID, "container")
    root3 = root2.find_element(By.ID, "main")

    shadow_root3 = expand_root_element(browser, root3)
    # shadow_root3 = expand_root_element(browser, root3)
    root4 = shadow_root3.find_element(By.CLASS_NAME, "showing-subpage")
    shadow_root4 = expand_root_element(browser, root4)
    root5 = shadow_root4.find_element(By.ID, "advancedPage")
    root6 = root5.find_element(By.TAG_NAME, "settings-privacy-page")
    shadow_root6 = expand_root_element(browser, root6)
    root7 = shadow_root6.find_element(By.ID, "pages")
    root8 = root7.find_element(By.TAG_NAME, "settings-subpage")
    root9 = root8.find_element(By.TAG_NAME, "site-details")
    shadow_root9 = expand_root_element(browser, root9)
    root10 = shadow_root9.find_element(By.ID, "plugins")
    shadow_root10 = expand_root_element(browser, root10)
    root11 = shadow_root10.find_element(By.ID, "permission")
    Select(root11).select_by_value("allow")

## Finally - Play It

In [20]:
browser.get(cctv_url)

In [21]:
raise Exception("stop here")

Exception: stop here

In [22]:
browser.quit()

In [ ]:
_home = os.getenv("HOME")
print(_home)

In [ ]:
browser.switch_to_window(browser.window_handles[1])

In [ ]:
browser.close()

In [ ]:
browser.switch_to_window(browser.window_handles[0])

In [ ]:
browser.current_url